# GPS RPi Integration

This Python code to be executed on the RedPitaya CPU is to retrieve from the JSON API running on the Raspberry Pi zero the GPS timestamp, location and APD ADC value. The events are syncronized among detectors using the hit_timestamp value.

In [ ]:
import os
import json
import requests

def retrieve_json_packet(hit_timestamp):
    url = 'http://192.168.2.3/json_api' 
    params = {'hit_timestamp': hit_timestamp}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_packet = response.json()
        if {'gps_timestamp', 'gps_pos', 'APD_ADC', 'date_time', 'hit_id'}.issubset(json_packet.keys()):
            return json_packet
        else:
            raise ValueError("JSON packet is missing one or more required fields")
    else:
        raise requests.exceptions.HTTPError(f"HTTP error {response.status_code}")

def save_json_to_file(json_data, folder_path, filename):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'w') as f:
        json.dump(json_data, f)


def retrieveEventFromRPIzero(hit_timestamp):
    json_packet = retrieve_json_packet(hit_timestamp)
    save_json_to_file(json_packet, 'rpi_zero_data', f'{hit_timestamp}.json')
